<a href="https://colab.research.google.com/github/gabrieljoseos/AnaliseCovid19/blob/main/Proj_AnaliseCovid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **Exercícios**

Este *notebook* deve servir como um guia para **você continuar** a construção da sua própria análise exploratória de dados interativa. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/) e seu *dashboard* [Google Data Studio](https://datastudio.google.com/).

---

# **COVID Dashboard**

## 1\. Contexto

A Covid-19, causada pelo coronavírus SARS-CoV-2, é uma doença respiratória que se espalhou rapidamente pelo mundo, resultando em uma pandemia global declarada pela Organização Mundial da Saúde (OMS) em março de 2020.


---

* Base de Dados sobre Casos de Covid-19

Os dados sobre casos da COVID-19 são compilados pelo centro de ciência de sistemas e engenharia da universidade americana John Hopkins. Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.).

* Base de Dados sobre Vacinação

Os dados sobre vacinação da COVID-19 são compilados pelo projeto Nosso Mundo em Dados (Our World in Data ou OWID) da universidade britânica de Oxford. Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de países.

Importância da Análise de Dados
Analisar os dados de vacinação e casos de Covid-19 é crucial para entender a dinâmica da pandemia.
Para este caso:
* Identificação de Padrões e Tendências
* Informação Pública

Portanto, a coleta e a análise de dados sobre vacinação e casos de Covid-19 são essenciais compreender como andou a vacinação nos estados e se há bons idicadores de contra casos de Covid-19.


## 2\. Pacotes e bibliotecas

In [1]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
...

Ellipsis

# 3. Casos de Covid-19

## 3\.1 Extração

In [2]:
# faça o código de extração dos dados:
#
# - coleta de dados;
# - wrangling da estrutura;
# - exploração do schema;
# - etc.


Carregamento da base de dados

In [3]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [4]:
cases.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605


O dado está compilado em um arquivo por dia, exemplo para 2021/12/01. Portanto, é necessário iterar dentro de um intervalo de tempo definido para extraí-lo.

In [5]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [6]:
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

Selecionada as colunas de interesse para análise no Brasil

In [7]:
cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = pd.concat([cases, case], ignore_index=True) # Use pd.concat to append DataFrames

## 3\.2 Transformação

Visualização do DataFrame após a seleção dos dados

In [8]:
cases.head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


In [9]:
cases.shape

(9828, 6)

In [10]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  9828 non-null   object        
 1   Country_Region  9828 non-null   object        
 2   Confirmed       9828 non-null   int64         
 3   Deaths          9828 non-null   int64         
 4   Incident_Rate   9828 non-null   float64       
 5   Date            9828 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 460.8+ KB


Trocando o nome das colunas de País e Estado para fácil entendimento.

In [11]:
cases = cases.rename(columns={'Province_State': 'state','Country_Region': 'country','Date':'date','Confirmed':'confirmed','Deaths':'deaths','Incident_Rate':'incident_rate'})

In [12]:
cases.head(1)

,state,country,confirmed,deaths,incident_rate,date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01


Substituindo o nome dos das cidades adicionando os pontos faltantes.

In [13]:
states_map = {'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

### 3.2.1 Enrquicimento

#### Adicionando chaves temporais

In [14]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

#### Adicionando coluna sobre população estimada do estado

In [15]:
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

In [18]:
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.75:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status


for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    # Use pandas.concat instead of .append
    cases_ = pd.concat([cases_, cases_per_state], ignore_index=True)

cases = cases_
cases_ = None

Visualização do tipo de cada coluna.

In [19]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   state                             9828 non-null   object        
 1   country                           9828 non-null   object        
 2   confirmed                         9828 non-null   int64         
 3   deaths                            9828 non-null   int64         
 4   date                              9828 non-null   datetime64[ns]
 5   month                             9828 non-null   object        
 6   year                              9828 non-null   object        
 7   population                        9828 non-null   float64       
 8   confirmed_1d                      9801 non-null   float64       
 9   confirmed_moving_avg_7d           9639 non-null   float64       
 10  confirmed_moving_avg_7d_rate_14d  9261 non-null 

## 3\.3 Carregamento

In [20]:
cases.to_csv('./covid-cases.csv', sep=',', index=False)

# 4. Vacinação

## 4\.1 Extração

Extração do banco de dados adiconando na quarta coluna formato de data para facilitar leitura no Looker Studio

In [21]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

<ipython-input-21-608e6d2dd56d>:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)


In [22]:
vaccines.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


Selecionando as colunas de interesse e renomeando para facilitar a leitura no Looker Studio

In [32]:
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]
vaccines = vaccines.rename(columns={'location': 'country'})

UndefinedVariableError: name 'location' is not defined

## 4\.2 Transformação

Validar o DataFrame



In [25]:
vaccines.head()

,country,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-09


In [28]:
vaccines.shape

(1625, 7)

In [29]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1625 entries, 0 to 1624
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   country                  1625 non-null   object        
 1   population               1625 non-null   float64       
 2   total_vaccinations       695 non-null    float64       
 3   people_vaccinated        691 non-null    float64       
 4   people_fully_vaccinated  675 non-null    float64       
 5   total_boosters           455 non-null    float64       
 6   date                     1625 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 89.0+ KB


Valores nulos apresentados, portanto será necessário ultilziar o ffill para preencher essas lacunas com os dado anterior válido.

In [30]:
vaccines = vaccines.fillna(method='ffill')

Criado filtro de data para padronizar com o DataFrame "cases".

In [31]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

Renomear o resto das colunas para facilitar o entendimento

In [33]:
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

### 4.2.1 Enriquecimento

#### Adicionando chaves temporais

In [34]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

Adicionando dados relativos

In [35]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)
# Feito o arredondamento dos valores para 4 casas decimais

Validar tipo dos dados

In [36]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   country           365 non-null    object        
 1   population        365 non-null    float64       
 2   total             349 non-null    float64       
 3   one_shot          349 non-null    float64       
 4   two_shots         330 non-null    float64       
 5   three_shots       121 non-null    float64       
 6   date              365 non-null    datetime64[ns]
 7   month             365 non-null    object        
 8   year              365 non-null    object        
 9   one_shot_perc     349 non-null    float64       
 10  two_shots_perc    330 non-null    float64       
 11  three_shots_perc  121 non-null    float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 34.3+ KB


Reorganizar as colunas da tabela

In [37]:
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

In [40]:
vaccines.tail(2)

,date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,month,year
363,2021-12-30,Brazil,215313504.0,331164041.0,166185628.0,0.7718,143398692.0,0.6660,26507937.0,0.1231,2021-12,2021
364,2021-12-31,Brazil,215313504.0,331273910.0,166195505.0,0.7719,143436012.0,0.6662,26571077.0,0.1234,2021-12,2021


## 4\.3 Carregamento

Afim de garantir a persisntencia no tipo do dado, realizado *type casting* das colunas.

In [41]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

In [42]:
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)

## 5. Visualização


[Dashboard](https://lookerstudio.google.com/reporting/89c866b5-325e-4756-9469-c0cea87164b1) realizado com os dados extraidados e manipulados por Python.